In [6]:
import pandas as pd
import numpy as np
np.set_printoptions(linewidth = 200)
nfl_2024_schedule = pd.read_csv("nfl_2024_thru_wk_04.csv")
wk_away_home_info = nfl_2024_schedule.loc[:, ['week', 'away_team', 'away_score', 'home_team', 'home_score']]
week_03 = wk_away_home_info[0:64]
teams = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 
         'LAC', 'LA', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

d = {}
for v, k in enumerate(teams):
    d[k] = v

df = week_03
s1 = df.shape
M1 = np.zeros([ s1[0], 33 ])

for i in range(s1[0]):
    M1[i, 32] = np.abs(df.iloc[i, 2] - df.iloc[i, 4])
    if df.iloc[i, 2] > df.iloc[i, 4]:
        M1[i, d[ df.iloc[i, 1]] ], M1[i, d[ df.iloc[i, 3]] ] = 1, -1
    elif df.iloc[i, 2] < df.iloc[i, 4]:
        M1[i, d[ df.iloc[i, 1]] ], M1[i, d[ df.iloc[i, 3]] ] = -1, 1
    else:
        M1[i, d[ df.iloc[i, 1]] ], M1[i, d[ df.iloc[i, 3]] ] = 1, 1

s = np.array(M1.shape)
p1 = M1[:, -1]
M1 = M1[:, 0:-1]

b = np.ones( s[1] - 1 ) + ( 0.5*np.sum( M1, axis=0 ) )

sm1 = np.array((M1.T@M1).shape)

p = M1.T@p1
p[-1,] = 0

M = M1.T@M1

C = M+2*np.identity(32);

M[-1,:] = np.ones(sm1[1])

r = np.linalg.lstsq(M, p, rcond=None)[0]

df3 = pd.DataFrame(
    {
        "Team" : teams,
        "Massey_ranking" : r
    }
)
df3_sorted_massey = df3.sort_values(by = "Massey_ranking", ascending = False, ignore_index = True)

rc = np.linalg.solve(C,b)
rc1 = np.linalg.inv(C.T@C)@C.T@b

df3 = pd.DataFrame(
    {
        "Team" : teams,
        "Colley_ranking" : rc1
    }
)

df3_sorted_cooley = df3.sort_values(by = "Colley_ranking", ascending = False, ignore_index = True)

In [8]:
df3_sorted_massey

,Team,Massey_ranking
0,MIN,15.100071
1,NO,14.669328
2,TB,11.934954
3,PIT,11.320676
4,DEN,8.846401
5,GB,8.710406
6,PHI,8.373498
7,KC,8.353711
8,ATL,8.179303
9,SF,6.404998


In [10]:
df3_sorted_cooley

,Team,Colley_ranking
0,MIN,0.843343
1,KC,0.840859
2,TB,0.745728
3,PIT,0.718623
4,DET,0.682298
5,HOU,0.667691
6,SEA,0.644144
7,WAS,0.636725
8,ATL,0.600899
9,DEN,0.593007
